# SageMaker Inference Recommender for Instant Recommendations

This notebook demonstrates how to get instant deployment recommendations from the DescribeModel API

The DescribeModel API can be found here: https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeModel.html

**This sample is provided for demonstration purposes, make sure to conduct appropriate testing if deriving this code for your own use-cases!**

## Set Up

In [12]:
!pip install boto3 --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [15]:
import time
import boto3
import sagemaker
import pandas as pd
from uuid import uuid4
from datetime import datetime, timedelta

In [16]:
sm_client = boto3.client(service_name="sagemaker")

## Step 1: Create the Model

For this sample notebook, we will use a simple model specifying only a Tensorflow DLC

In [17]:
model_name = "instant-rec-demo-nb-model-" + str(uuid4())
print(model_name)

instant-rec-demo-nb-model-c597a884-cb23-462b-a0e5-9ebfcdf9126f


In [18]:
region = boto3.Session().region_name
image = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="1.12.0",
    image_scope="inference",
    instance_type="ml.c5.4xlarge",
)
print(image)

520713654638.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu


In [19]:
execution_role = sagemaker.get_execution_role()

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=execution_role, Containers=[{"Image": image}]
)

print(create_model_response)

{'ModelArn': 'arn:aws:sagemaker:us-west-2:337323248444:model/instant-rec-demo-nb-model-c597a884-cb23-462b-a0e5-9ebfcdf9126f', 'ResponseMetadata': {'RequestId': 'a2c3c7ca-937f-4e52-a51b-6df310c7254f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a2c3c7ca-937f-4e52-a51b-6df310c7254f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '124', 'date': 'Thu, 25 May 2023 16:33:17 GMT'}, 'RetryAttempts': 0}}


## Step 2: Describe the Model

Describe the Model until `DeploymentRecommendations` goes into a `COMPLETED` state. On average, you should expect to have DeploymentRecommendations available within a second

In [20]:
completed = timed_out = False

start = datetime.today()
max_duration = start + timedelta(minutes=1)

while not completed and not timed_out:
    describe_model_response = sm_client.describe_model(ModelName=model_name)
    deployment_recommendation = describe_model_response.get("DeploymentRecommendation")

    completed = (
        deployment_recommendation is not None
        and "COMPLETED" == deployment_recommendation.get("RecommendationStatus")
    )

    time.sleep(0.5)
    timed_out = datetime.today() > max_duration

    print(datetime.today(), "DeploymentRecommendation:", deployment_recommendation)

2023-05-25 16:33:19.899569 DeploymentRecommendation: {'RecommendationStatus': 'IN_PROGRESS'}
2023-05-25 16:33:20.515336 DeploymentRecommendation: {'RecommendationStatus': 'COMPLETED', 'RealTimeInferenceRecommendations': [{'RecommendationId': 'instant-rec-demo-nb-model-c597a884-cb23-462b-a0e5-9ebfcdf9126f/xkEZXE8o', 'InstanceType': 'ml.g4dn.2xlarge', 'Environment': {}}, {'RecommendationId': 'instant-rec-demo-nb-model-c597a884-cb23-462b-a0e5-9ebfcdf9126f/E7rwqPBa', 'InstanceType': 'ml.c6i.large', 'Environment': {}}, {'RecommendationId': 'instant-rec-demo-nb-model-c597a884-cb23-462b-a0e5-9ebfcdf9126f/5PoiMLHb', 'InstanceType': 'ml.c5.2xlarge', 'Environment': {}}]}


## Step 3: Visualize the DeploymentRecommendations

In [21]:
pd.set_option("display.max_colwidth", None)
df = pd.DataFrame(
    deployment_recommendation.get("RealTimeInferenceRecommendations"),
    columns=["RecommendationId", "InstanceType", "Environment"],
)
display(df)

,RecommendationId,InstanceType,Environment
0,instant-rec-demo-nb-model-c597a884-cb23-462b-a0e5-9ebfcdf9126f/xkEZXE8o,ml.g4dn.2xlarge,{}
1,instant-rec-demo-nb-model-c597a884-cb23-462b-a0e5-9ebfcdf9126f/E7rwqPBa,ml.c6i.large,{}
2,instant-rec-demo-nb-model-c597a884-cb23-462b-a0e5-9ebfcdf9126f/5PoiMLHb,ml.c5.2xlarge,{}


## Step 4: Cleanup

In [22]:
sm_client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '9bf56669-4f92-4803-97f8-f2bd3fdb193f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9bf56669-4f92-4803-97f8-f2bd3fdb193f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 25 May 2023 16:33:23 GMT'},
  'RetryAttempts': 0}}